In [ ]:
%tensorflow_version 2.x 

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)



Running on TPU  ['10.70.40.146:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.70.40.146:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.70.40.146:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [ ]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In case you want to upload your own play or poem uncomment the next cell.

In [ ]:
# from google.colab import files
# path_to_file = list(files.upload().keys())[0]

In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


A function that can convert our numeric values to text.


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


###Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

Our first step will be to create a stream of characters from our text data.

In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

Next we can use the batch method to turn this stream of characters into batches of desired length.

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           16640     
_________________________________________________________________
lstm_3 (LSTM)                (64, None, 1024)          5246976   
_________________________________________________________________
dense_3 (Dense)              (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.7214168e-03 -3.7664408e-03 -9.1661699e-04 ...  1.8459468e-03
   -8.5995259e-04 -4.0344526e-03]
  [-2.1083646e-03 -1.9875665e-03 -2.8220646e-03 ... -1.5756772e-03
   -3.3829734e-04  2.1011010e-03]
  [ 2.4926802e-04 -4.7568800e-03  5.3149182e-03 ... -2.0274357e-03
   -2.5388990e-03  5.3489325e-03]
  ...
  [ 9.3990918e-03  3.2796932e-03  1.8464159e-03 ...  9.9818259e-03
    6.2457146e-03  5.4289810e-03]
  [ 6.8903407e-03 -2.7606165e-04  1.1375048e-03 ...  7.3290560e-03
    1.0305019e-03  1.5952555e-04]
  [ 6.7552011e-03  4.3923529e-03  2.5722980e-03 ...  4.1242093e-03
   -4.1194120e-03 -6.5245242e-03]]

 [[ 3.7104872e-03  3.8156428e-03 -1.3001147e-03 ... -2.2662883e-03
    1.6618258e-03 -3.0975416e-04]
  [ 3.6954600e-04 -1.5382746e-03 -1.5482299e-03 ...  6.9325499e-04
    1.6306137e-04 -3.9305640e-03]
  [ 4.6461932e-03 -2.0615358e-03  2.9857364e-04 ...  4.6070791e-03
    3.8013333e-03 -5.7799146e-03]
  ...
  [ 1.3278862e-03  8.8257883e-03  2.9678859e-03 ...  1.8223186e

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00272142 -0.00376644 -0.00091662 ...  0.00184595 -0.00085995
  -0.00403445]
 [-0.00210836 -0.00198757 -0.00282206 ... -0.00157568 -0.0003383
   0.0021011 ]
 [ 0.00024927 -0.00475688  0.00531492 ... -0.00202744 -0.0025389
   0.00534893]
 ...
 [ 0.00939909  0.00327969  0.00184642 ...  0.00998183  0.00624571
   0.00542898]
 [ 0.00689034 -0.00027606  0.0011375  ...  0.00732906  0.0010305
   0.00015953]
 [ 0.0067552   0.00439235  0.0025723  ...  0.00412421 -0.00411941
  -0.00652452]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[-2.7214168e-03 -3.7664408e-03 -9.1661699e-04 -5.6881504e-03
 -3.0392020e-03  5.5113225e-04  4.7696992e-03  1.4358988e-03
 -2.1405253e-03 -7.2176964e-03  5.3794533e-03 -4.2288741e-03
 -4.2580557e-03  4.5794304e-03  2.1420338e-03 -1.9566454e-03
 -4.8860507e-03  2.9685160e-03 -2.6448620e-03 -2.9670887e-03
 -2.3979738e-03 -1.1447261e-03 -2.3719885e-03 -6.2657003e-03
 -1.4892817e-03  4.5522768e-03  3.1369578e-03  1.2221790e-03
 -3.8037107e-03  1.1465633e-03 -2.8878306e-03  6.5641664e-04
 -6.4515078e-04 -5.6674425e-04  3.3650040e-03  1.2322051e-03
 -1.6582822e-03 -2.6544614e-05  6.6020852e-04  3.1055510e-03
  8.9232117e-04  3.7697286e-03  3.3389763e-03 -4.8741866e-03
  1.7530983e-04 -2.7147722e-03  3.6514599e-03 -9.1619347e-04
  3.7534866e-03 -4.6814983e-03 -5.7719056e-03 -6.1963795e-04
 -3.8799020e-03 -8.1379190e-03 -2.4105161e-03  3.8194661e-03
 -7.6545664e-04  4.1894051e-03  3.9657718e-04  2.6861273e-04
  5.0058961e-03 -2.5667991e-03  1.8459468e-03 -8.5995259e-04
 -4.034452

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

';evzzaLmVGjVHie;uvnNikTj!EpmujMPAh??!KyeS.Vy&dt.v3m$MO;\npe!bQuva?Tzfgh:bAl$V.b!isQ-cyw!xIrPIIF!Dzw;f'

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
with tpu_strategy.scope():
  history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50


ValueError: ignored

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
checkpoint_num = 10
model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

Generating text

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))